<a href="https://colab.research.google.com/github/mcnica89/Markov-Chains-RL-W24/blob/main/Markov_Chains_Lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np

# Problem 1 Simulate a Markov Chain given the Transition Matrix

Given an integer $n \geq 1$, an initial state $x_{init} \in \{0,1,\ldots,n\}$, an $n+1 \times n+1$ transition matrix, and a final time $t_{max} \in \mathbb{N}$, write a program that simulates the Markov Chain on the state space $\{0,1,\ldots,n\}$, i.e. produce a sequence of random variables $X_0,X_1,\ldots, X_{t_{max}}$ and output this as a numpy vector of length $(t_{max}+1,)$.

In [ ]:
def markov_chain_sim(n,x_init,M,t_max):
  #Input:
  #  n : int >=1 . The state space for the markov chain is {0,1,2,...n}
  #      Note: The NUMBER of states is n+1 (since we start at 0). The MAXIMUM VALUE is n.
  #  x_init : int starting state at time 0
  #  M : a numpy arrray of size (n+1,n+1) which is the transition matrix
  #  t_max : number of time steps to simulate
  #Output:
  #  answer : a numpy array of size (t_max+1,) of int so that
  #          Notes:
  #          answer[0] = x_init always
  #          answer[t] is always in {0,1,2,...n} for all t
  #          The function is a **random** function and will have different outputs everytime you run it.

  answer = np.zeros(t_max+1,dtype=int)
  answer[0] = x_init
  #
  # Your code here
  #
  return answer

#Example: For the rainy day/nice day matrix from class, starting from a "Nice" day,
#n=1,transition_matrix = np.array([[0.4,0.6],[0.2,0.8]]), x_init=1,t_max=4 the program might return
# answer = [1, 1, 0, 1, 1] to mean that it came out "Nice","Nice","Rainy","Nice","Nice"


## Checking your answer to Problem 1

(You don't have to submit any code or antyhing for this section; this section just gives you a tool you can use to test your function.)


In [ ]:
#To check if your program is running as expected, we will take the output of your program, and compute the emprical transition matrix
# The provided function below computes teh empirical transition matrix.
# This is a matrix which uses the number of transitions from each state to approximate the transition matrix
# If your program is working as expected, and all states are reachable from all states, the empircal transition matrix should be close to the true transition matrix as t_max gets large

def empirical_transition_matrix(n, sequence):
  #Input:
  # n : Maximum value of state
  # sequence : a realization sequnece (X_0,X_1,...X_{t_max}) of the markov chain
  #Ouput:
  # answer : an (n+1,n+1) matrix which is the empircal transition matrix
  #          the emprical transition matrix whose i,j entry is the
  #          number of time (i,j) appears divided by the total number i appears.

  #counts the number of times each state is visited (not counting the last one!)
  state_counts = np.bincount(sequence[:-1])

  #counts the number of times each pair (state,next_state) happens
  # the states are encoded here by the formula (state,next_state) -> state + n_states*next_state
  # so for example if n_states = 2, then the pair (0,1) -> 2 is encoded as the number 2.
  n_states = n+1
  transitions = sequence[:-1] + n_states*np.roll(sequence,-1)[:-1]

  #counts the number of each type of transition using the same encoding
  #Ex: if n_states = 2, transition_counts[2] is the number of times (state,next_state)=(0,1) appears
  transition_counts = np.bincount(transitions)

  #will be the output to the function
  answer = np.zeros((n_states,n_states))

  def safe_divide(a,b):
    #returns a/b but is "safe" in the case b=0. Return 0 if b=0.
    # where a,b are numpy arrays, returns a/b if b!=0 and just 0 if b=0
    return np.divide(a, b, out=np.zeros_like(a,dtype=float), where=(b!=0) )

  #the answer is the numebr of time (i,j) appears over the number of times i appears
  i,j = np.indices((n_states,n_states))
  answer[i,j] = safe_divide(transition_counts[i+n_states*j],state_counts[i])

  #This above code with np.indices is a faster/more efficient version of the following equivalent code with for loops:
  #for i in range(n_states):
  #  for j in range(n_states):
  #    if state_counts[i] > 0:
  #      #the answer is the numebr of time (i,j) appears over the number of times i appears
  #      answer[i,j] = transition_counts[i+n_states*j]/state_counts[i]

  return answer


#Example:
#example_sequence = np.array([1,1,0,1,1],dtype=int)
#empirical_transition_matrix(n=1,sequence=example_sequence)
# This produces:
#
#array([[0.        , 1.        ],
#       [0.33333333, 0.66666667]])
# This is because in the given sample, 100% of the State0s are followed by State 1
# and 1/3 of the State1's are followed by a State0, and 2/3 of the State1s are followed by a State1.
#
#
#### HOW TO TEST YOUR FUNCTION
## FACT: If you have an irreducible Markov chain, the empirical transition matrix will converge to the true transition matrix as the length of the simulation grows large
## To test your simulation function, feed a long output into the empircal transition matrix function and see if you get back the original transition matirx.

# Warning for Problems 2 and 3

It is possible to approximately find the answers to the questions for Problems 2 and 3 using the simulation machine from Problem 1. (For example: to find the expected value of something, just run 1,000,000 simulations, and take the average value. This is called a Monte Carlo method).

Do *NOT* attempt to use simulations to answer the questions in Problems 2 and 3; it is very hard to ensure that you are within the desired accuracy (and could take 100s of hours of simualtions). If you try this, you will not get full marks and your code may also time out before it is graded! Instead, you should solve the problems using matrix/linear algebra type computations like we did in class.

You are however *HIGHLY* encouraged to use simualtions to check if your work makes sense. (For example: if you think the expected value is 8.6 and your run 100 simulations that average out to 8.55, this is a good sign your 8.6 answer makes sense)

# Problem 2: Easy21 from an "infinite deck" with probability calculations for a specific strategy

**Setup** Easy21 is a mathematical game that is similar to blackjack. The final project of the course will involve making an AI agent that plays Easy21. In this problem, you will calculate some probabilities/expected values related to Easy21.

Here are the rules for Easy21:
- Easy21 is played with a deck of cards that has 30 cards in it. There are 10 red cards with values [1,2,3,4,5,6,7,8,9,10] and 20 black cards with values [1,2,3,4,5,6,7,8,9,10] each of which appear twice. When adding cards, Black cards counts as POSITIVE their value, and RED cards count as NEGATIVE their value.
- For the final project, we will play Easy21 with finite decks of cards (so e.g. if all three 10s are dealt out, then you know there are no more 10s left in the deck). However, this Problem is about Easy21 from a "infinite deck" (or equivlently drawing cards "with replacement"). This means you can pretend each new card is taken from a dice roll from a 30 sided dice.
- Like in Blackjack, in Easy21 your objective is to have the highest possible total of your cards without exceeding 21. You can either "stick" and stay at your current total or "hit" to get more cards.
- You start the round with one face up card which is your starting value. The value of this card between $1$ to $10$ is your starting value (even if the card is a red card).
- You then choose to "hit" to get another cards until you decide to "stick" with the cards you have. You add up your cards to get your total. When adding cards, Black cards counts as POSITIVE values, and RED cards count as NEGATIVE. If the players sum exceeds 21, or becomes less than 1, then they "go bust" and lose the game immediatly.
- For $\ell \in \mathbb{N}$, the strategy "hit below $\ell$" is the strategy to always hit if your sum is $\leq \ell-1 $ and stick if your sum is $ \geq \ell$. (Example: For the final project, the dealer will always follows the "hit below 17" strategy with $\ell=17$.)

For a fixed value of $\ell$, when following the hit below $\ell$ strategy, the players total is a Markov chain on the state space [0,1,2...,21] where we intepret the State 0 to mean "player has busted by either going below 1 or above 21" at some point.


## Q2a)

Write a function that inputs $\ell$ and outputs the transition matrix for the "hit below l"-strategy Markov chain.

In [ ]:
def hit_below_l_transition_matrix(l):
  #Input:
  # l: an integer between 1 and 21
  #   This is the target value used in the "hit below l" strategy
  #Output:
  # answer : a numpy array of shape (22,22) of floats
  #          which represents the transition matrix on the state space {0,1,...,21}
  #         Note that state 0 represents the "busted" state which can happen if you go over 21 or below 1.

  answer = np.zeros((22,22),dtype=float)
  #
  # Your code here
  #
  return answer

## Q2b)


Write a function hit_below_l_probabs which inputs $\ell$ and the starting value $x_{init}$, the transition matrix $M$ from part a), and outputs a vector of length 22 with the probabilities to be in the state at the end of the hand. (Note: Because the player always hits below $\ell$, the answer should have 0's in all states except for State0, State $\ell$, State $\ell+1$,...State 21). Your answers should be correct to within 0.001% for each probability.



In [ ]:
def hit_below_l_probabs(l,x_init,M):
  #Input:
  # l: an integer between 1 and 21
  #   This is the target value used in the "hit below l" strategy
  # x_init: the starting value of our card.
  #      M: The transitiion matrix for the problem (the outout of M=hit_below_l_transition_matrix)
  #Output:
  # answer : a numpy array of shape (22,) of floats in (0,1)
  #          which represents the probabilites to be at any state after you finish
  #          Notes:
  #          The state 0 represents the "busted" state which can happen if you go over 21 or below 1.
  #          Since we always hit below l, the answer[i] ~= 0.0 should be true for all 1 <= i <= l-1

  answer = np.zeros(22,dtype=float)
  #
  # Your code here
  #
  return answer

## Q2c)

Write a function hit_below_l_E_cards which has the same inputs as Q2b) and outputs how many cards on average are dealt out before the round ends (either because the player choosed to "stick" or because they busted). Your answer should be correct to within 0.00001.

In [ ]:
def hit_below_l_E_cards(l,x_init,M):
  #Input:
  # l: an integer between 1 and 21
  #   This is the target value used in the "hit below l" strategy
  # x_init: the starting value of our card.
  #      M: The transitiion matrix for the problem (the outout of M=hit_below_l_transition_matrix)
  #Output:
  # answer : a float
  #          which represents the E number of cards dealt out before we stopped.
  #          Note that state 0 represents the "busted" state which can happen if you go over 21 or below 1.

  answer = 0.0
  #
  # Your code here
  #

  return answer

# Problem 3 Gamblers Ruin (with prize cars)

## Setup

A gambler with $x_{init}\in \mathbb{N}$ dollars arrives at a Casino and is determined to gamble until they either go broke or leave the Casino with a predetrmined amount $n \in \mathbb{N}$ dollars. They do this by placing a sequence of bets. Each bet has a chance $p_{win}$ for the Gambler to win and $1-p_{win}$ for the gambler to lose. The bet pays out at even odds, i.e. if they win they win exactly the amount they bet. The gambler keeps gambling until they go broke (i.e. their bankroll becomes $x=0$) OR until they reach the pre-defined target amount $n$ (i.e their bankroll becomes $x=n$).

*Bet size:* For every possible bankroll amount $x \in \{1,\ldots,n-1\}$, the gambler has a predefined bet size amount $b_x \in \mathbb{N}$. When the gambler has exactly $x$ dollars, they will place a bet of size $b_x$ dollars. The bet sizes must satify two conditions:
0. $b_x \geq 1$ (The gambler must be at least 1 dollar at each step)
1. $b_x \leq x$ (The bet size cannot exceed the current amount of money the gambler has)
2. $b_x \leq n-x$ (The gambler will never bet more than is needed to get to $n$ if they win.)



## Q3a)

The gambler's bankroll $X$ is a Markov chain on the state space $\{0,1,...,n\}$. Write a program to find the transition matrix. The program takes as input the target amount $n$, the win probability $p_{win}$, and the betting sizes vector $[0,b_1,\ldots,b_{n-1}]$ as a numpy vector (Note: The vector is padded with a zero at the beginning so that the i-th entry of the vector is $b_i$)

In [ ]:
def gamblers_ruin_matrix(n, p_win, bet_size):
  # Input:
  #  n : integer >= 1
  #      Target value for the Gambler
  #  p_win : float in [0,1]
  #          Win probability for each bet
  #  bet_size : numpy array of size (n,) of int
  #              bet_size[x] is the size of the bet placed when at x dollars
  #              Note: bet_size[0] is never used (only included to avoid annoying off-by-1 index errors)
  # Output:
  #  answer = numpy array of shape (n+1,n+1) of floats
  #           Transition probability matrix for the Markov chain on state space [0,1,..n]

  # Check that the bet sizes are all valid
  assert np.all(bet_size[1:] >= 1) #check bet_size[x]>=1 for all x>=1
  assert np.all(bet_size <= np.arange(n)) #check bet_size[x] <= x for all x
  assert np.all(bet_size <= n-np.arange(n)) #check bet_size[x] <= n-x for all x


  answer = np.zeros((n+1,n+1),dtype=float)
  #
  # Your solution goes here!
  #
  return answer

#Example Answer:
# When n=3, p_win=0.5 and bet_size=np.array([0,1,1])
# The correct answer is:
# example_answer = np.array([[1,0,0,0],[0.5,0,0.5,0],[0,0.5,0,0.5],[0,0,0,1]]
# whihc looks like:
# [[1.  0.  0.  0. ]
#  [0.5 0.  0.5 0. ]
#  [0.  0.5 0.  0.5]
#  [0.  0.  0.  1. ]]
# you can check your answer in this case by using np.allclose(answer,example_answer)


##Q3b)

**Setup**
If the gambler can reach their goal of a bankroll with $n$ dollars, they recieve a special *one-time* prize of $1$ new car as a fun bonus prize from the Casino. For each $t$, let $\vec{v}^{(t)}$ be the vector (which we think of as a column vector) of size $n+1$ whose $x$-th entry represents the expected number of prize cars they have won after $t$ rounds of betting starting from the initial state $x$. i.e.

$$v^{(t)}_x := \mathbb{E}[ \text{ Number of Prize Cars Won in first $t$ steps } | X_0 = x]$$

For example, $\vec{v}^{(0)}$ is the vector $\vec{v}^{(0)}=\begin{bmatrix} 0 \\ 0 \\ \vdots \\0 \\1\end{bmatrix}$, (all zeros and a single 1 in position $n$) because the only way they won the car before any steps have been played is if they started at the final goal $X_0=n$. (Reminder: The prize car is a special one-time prize; at any given time the Gamvler has either 0 prize cars or exactly 1 prize car)

**Question**
Q3b) Write a function which inputs the value $n$, the $(n+1,n+1)$ transition $M$ from part a), and the number of steps $t$ and which outputs the vector $\vec{v}^{(t)}$ of shape $(n+1,)$. (Hint: Once you figure out what is going on in this question, the actual program to do this is very short and simple. This question is trying to get you to do some thinking about what the formula is and where it comes from.)


In [ ]:
def gamblers_ruin_E_prize_cars(n,t,M):
  #Input:
  #  n : integer >= 1
  #      Target value for the Gambler
  #  t : integer >= 0,
  #      which time value (number of steps) to calcualte for
  #  M : numpy array of shape (n+1,n+1) which is the transition matrix for the gambler from Q3a
  #Output:
  #  answer : vector of shape (n+1,) of floats
  #           represents the vector v^{(t)}

  answer = np.zeros(n+1,dtype=float)
  #
  # Your code here!
  #
  return answer

## Q3c)

The limit $v^\infty_{x} := \lim_{t \to \infty} v_x^{(t)}$ exists and has a real life interpretation of something that the Gambler would care about which can be explained to the Gambler without referencing the prize car. (i.e. even if the Casino is not giving away a prize car $v^\infty_{x}$ has a nice explanation)

Figure out what the "simple" explanation of this vector is, and then fill in the blank on this sentence: "Starting from an initial bankroll of state $x$, $v_x^\infty$ represents _________ ". A human (the prof!) will read and grade your answer for this one.

In [ ]:
def gamblers_ruin_explanation():
  preamble = "Starting from the initial value x, v_x^\infty represents "

  fill_in_blank_answer = "..." #Fill in the blank here by typing in this string!

  full_sentence = preamble + fill_in_blank_answer

  return fill_in_blank_answer

BONUS QUESTION
Q3d) Prove that for any $n$ and for *any* valid betting strategy, if $p_{win}=\frac{1}{2}$ then the limit is exactly:
$$v^{\infty}_x = \frac{x}{n}$$

(Hint: A possible proof is to find an equation that the limit must satisfy and then show that $x/n$ must be the only solution to that equation.)

(Note: If you are not able to prove the result for *any* valid betting strategy, you can recieve partial credit for proving it for only the min-bet betting strategy of always betting $b_x = 1$. You must still prove it holds for any $n$.)


To submit an answer to this bonus question you should type your solution out in LaTeX and submit it as pdf to the CourseLink dropbox site.
